# CNN を用いた画像分類モデルの作成

このノートブックでは、TensorFlow 2.x を使用して CNN の画像分類モデルを作成する方法を学びます

MNIST の数の分類をするための、シンプルな畳み込みニューラルネットワーク (CNN: Convolutional Neural Network) の学習について説明します。このシンプルなネットワークは MNIST テストセットにおいて、99%以上の精度を達成します。



## 目的
- CNN の画像分類モデルの作り方を理解する

## 環境の準備

In [ ]:
import matplotlib.pyplot as plt
import logging

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds
print("TensorFlow version: ",tf.version.VERSION)

# set TF error log verbosity
logging.getLogger("tensorflow").setLevel(logging.INFO)

## MNISTデータセットのダウンロードと準備
`tfds` モジュールを使用して、`tf.data` 形式のパブリックデータをダウンロードします

`tfds` で使用できるデータの一覧は[こちら](https://www.tensorflow.org/datasets/catalog/overview)を参照ください


In [ ]:
datasets, info = tfds.load('mnist',data_dir='./data', with_info=True, as_supervised=True)

BATCH_SIZE = 128
train_ds = datasets['train'].batch(BATCH_SIZE).prefetch(1)
test_ds = datasets['test'].batch(BATCH_SIZE).prefetch(1)

In [ ]:
print('train data: {}'.format(train_ds))
print('test data: {}'.format(test_ds))

## CNN モデルの作成

### 畳み込みの基礎部分の作成

下記のコードは、一般的なパターンで畳み込みの基礎部分（Conv2D と MaxPooling2D レイヤーのスタック）を定義しています。

入力として、CNN は shape (image_height, image_width, color_channels) のテンソルをとります。<br>
color channels について、MNIST は1つ (画像がグレースケールのため) の color channels がありますが、カラー画像には3つ (R, G, B) があります。この例では、MNIST 画像のフォーマットである shape (28, 28, 1) の入力を処理するように CNN を構成します。これを行うには、引数 input_shape を最初のレイヤーに渡します。

In [ ]:
inputs = tf.keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
conv_output = layers.Conv2D(64, (3, 3), activation='relu')(x)

### Dense レイヤーを追加

モデルを完成するために、(shape (3, 3, 64) の) 畳み込みの基礎部分からの最後の出力テンソルを、1つ以上の Dense レイヤーに入れて分類を実行します。現在の出力は 3D テンソルですが、Dense レイヤーは入力としてベクトル (1D) を取ります。まず、3D 出力を 1D に平滑化 (または展開) してから、最上部に1つ以上の Dense レイヤーを追加します。MNIST は 10 個の出力クラスを持ちます。そのため、我々は最後の Dense レイヤーの出力を 10 にし、softmax関数を使用します。

In [ ]:
x = layers.Flatten()(conv_output)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

では、完成したモデルを可視化してみましょう

In [ ]:
model = models.Model(inputs, output, name='cnn')
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, 'cnn_model.png', show_shapes=False)

## モデルのコンパイルと学習

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, epochs=5)

## モデルの精度をテストデータを使用して確認する

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

99% 程度の精度を達成することができました。

### 学習済みモデルを使用した推論

では、 `model.predict()` を使用してテストデータに対する推論結果を確認してみましょう

In [ ]:
y_hat = model.predict(test_ds).argmax(axis=1)

In [ ]:
num_show = 5

for data in test_ds.take(1):
    for i, d in enumerate(data[0]):
        plt.imshow(tf.reshape(d, (28, 28)))
        plt.axis('off')
        plt.title("Prediction: {}".format(str(y_hat[i])))
        plt.show()
        if i == num_show - 1:
            break